In [1]:
# pip install numpy-stl

Note: you may need to restart the kernel to use updated packages.


In [6]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import struct
import os
from scipy.interpolate import griddata
from scipy.signal import find_peaks
from scipy import stats
import seaborn as sns
import pandas as pd

plt.rcParams.update({
    'font.size': 16,
    'font.family': 'Arial'
})

def read_stl(file_path):
    with open(file_path, 'rb') as file:
        header = file.read(80)
        num_triangles = struct.unpack('<I', file.read(4))[0]
        vertices = []
        volume = 0
        for _ in range(num_triangles):
            file.read(12)  # Normal vector
            v1 = struct.unpack('<fff', file.read(12))
            v2 = struct.unpack('<fff', file.read(12))
            v3 = struct.unpack('<fff', file.read(12))
            vertices.extend([v1, v2, v3])
            file.read(2)  # Attribute byte count

            # Calculate the volume of the triangular prism
            a = np.array(v1)
            b = np.array(v2)
            c = np.array(v3)
            volume += np.abs(np.dot(a, np.cross(b, c))) / 6

        return np.array(vertices), volume


def align_track(points):
    mean = np.mean(points, axis=0)
    centered_points = points - mean
    pca = PCA(n_components=3)
    pca.fit(centered_points)
    principal_axes = pca.components_

    # Ensure the determinant is positive for a right-handed coordinate system
    if np.linalg.det(principal_axes) < 0:
        principal_axes[2] = -principal_axes[2]

    # Assuming the third principal axis should be pointing 'up'
    # Check if it's pointing 'down' instead
    if principal_axes[2, 2] < 0:
        # If the Z component of the third principal axis is negative,
        # it means the track is upside down. Flip it to correct the orientation.
        principal_axes[2] = -principal_axes[2]

    aligned_points = centered_points @ principal_axes.T
    return aligned_points

def create_grid_and_find_max_z(points, grid_size):
    x_min, x_max = np.min(points[:, 0]), np.max(points[:, 0])
    y_min, y_max = np.min(points[:, 1]), np.max(points[:, 1])
    
    # Adding small margins to avoid cutting off the edges
    x_margin = (x_max - x_min) * 0.01
    y_margin = (y_max - y_min) * 0.01
    
    x_grid = np.linspace(x_min - x_margin, x_max + x_margin, grid_size)
    y_grid = np.linspace(y_min - y_margin, y_max + y_margin, grid_size)
    X, Y = np.meshgrid(x_grid, y_grid)
    
    Z = griddata(points[:, :2], points[:, 2], (X, Y), method='nearest')
    
    return X, Y, Z

def find_kde_slope_threshold(z_values, percent):
    sns.kdeplot(z_values, fill=True)
    kde = sns.kdeplot(z_values)
    kde_lines = kde.get_lines()[0]
    x, y = kde_lines.get_data()
    
    # Compute the derivative of the KDE
    dydx = np.gradient(y, x)
    
    # Find peaks in the KDE
    peaks, _ = find_peaks(y)
    if len(peaks) == 0:
        raise ValueError("No peaks found in KDE")
    
    # Filter peaks to consider only those with z values less than 0
    negative_peaks = [peak for peak in peaks if x[peak] < 0]
    if len(negative_peaks) == 0:
        raise ValueError("No peaks found with z values less than 0")
    
    # Identify the largest peak among the negative peaks
    largest_peak_index = max(negative_peaks, key=lambda peak: y[peak])
    peak_value = y[largest_peak_index]
    
    # Find the zero-crossing points after the largest peak
    zero_crossings = np.where(np.diff(np.sign(dydx)))[0]
    zero_crossings_after_peak = zero_crossings[zero_crossings > largest_peak_index]
    if len(zero_crossings_after_peak) == 0:
        raise ValueError("No zero-crossing found past the largest peak")
    
    bottom_peak_index = zero_crossings_after_peak[0]
    bottom_peak_value = x[bottom_peak_index]
    
    # Calculate the Z threshold as a specified percentage between the peak and the bottom of the peak
    z_threshold = x[largest_peak_index] + percent / 100.0 * (bottom_peak_value - x[largest_peak_index])
    
    plt.close()
    return z_threshold


def calculate_surface_roughness(Z):
    # Remove NaNs for calculation
    Z = Z[~np.isnan(Z)]
    
    # Arithmetic Mean Deviation (Ra)
    Ra = np.mean(np.abs(Z - np.mean(Z)))

    # Root Mean Square Deviation (Rq)
    Rq = np.sqrt(np.mean(Z**2))

    # Maximum Profile Peak Height (Rp)
    Rp = np.max(Z)

    # Maximum Profile Valley Depth (Rv)
    Rv = np.min(Z)

    # Ten-Point Mean Roughness (Rz)
    peaks = np.sort(Z)[-5:]
    valleys = np.sort(Z)[:5]
    Rz = np.mean(peaks+valleys)

    # Skewness (Rsk)
    Rsk = (np.mean((Z - np.mean(Z))**3)) / (np.std(Z)**3)

    # Kurtosis (Rku)
    Rku = (np.mean((Z - np.mean(Z))**4)) / (np.std(Z)**4)

    return Ra, Rq, Rp, Rv, Rz, Rsk, Rku

def calculate_widths_and_heights(X, Y, Z, sections=1000):
    x_min, x_max = np.min(X), np.max(X)
    section_width = (x_max - x_min) / sections

    widths = []
    heights = []

    for i in range(sections):
        x_start = x_min + i * section_width
        x_end = x_start + section_width

        z_section = Z[(X >= x_start) & (X < x_end)]
        y_section = Y[(X >= x_start) & (X < x_end)]

        if np.any(~np.isnan(z_section)):
            y_min_edge = np.min(y_section[~np.isnan(z_section)])
            y_max_edge = np.max(y_section[~np.isnan(z_section)])

            width = y_max_edge - y_min_edge
            widths.append(width)

            height = np.nanmax(z_section)
            heights.append(height)

    mean_height = np.mean(heights)
    z_scores_heights = stats.zscore(heights)
    
    threshold = 0.5  # Define your outlier threshold
    
    outlier_indices_heights = np.where(abs(z_scores_heights) > threshold)
    outlr_idx_h = len(outlier_indices_heights[0])

    # Calculate surface roughness parameters for the entire Z array
    Ra, Rq, Rp, Rv, Rz, Rsk, Rku = calculate_surface_roughness(Z)

    return widths, heights, np.max(widths), np.max(heights), outlr_idx_h, Ra, Rq, Rp, Rv, Rz, Rsk, Rku

def calculate_volume(X, Y, Z, sections=20):
    x_min, x_max = np.min(X), np.max(X)
    section_width = (x_max - x_min) / sections

    volume = 0

    for i in range(sections):
        x_start = x_min + i * section_width
        x_end = x_start + section_width

        z_section = Z[(X >= x_start) & (X < x_end)]
        y_section = Y[(X >= x_start) & (X < x_end)]

        if np.any(~np.isnan(z_section)):
            # Find the Y values at the edges of the contour
            y_min_edge = np.min(y_section[~np.isnan(z_section)])
            y_max_edge = np.max(y_section[~np.isnan(z_section)])

            width = y_max_edge - y_min_edge
            height = np.nanmax(z_section)  # Assuming height is the max Z value in the section

            # Calculate the radius as half of the height
            radius = height / 2

            # Calculate the area of the semi-circle and then the volume of the section
            area = 0.5 * np.pi * radius**2
            volume = area * section_width
            volume += volume

    return volume


def plot_projections(file_path, grid_size=1000, percent=75):
    points, volume = read_stl(file_path)
    aligned_points = align_track(points)

    # Find the Z threshold from KDE
    z_values = aligned_points[:, 2]
    z_threshold = find_kde_slope_threshold(z_values, percent)


    X, Y, Z = create_grid_and_find_max_z(aligned_points, grid_size)

    # Apply Z threshold
    Z[Z < z_threshold] = np.nan  # Set values below threshold to NaN
    # Add the absolute value of the z_threshold to each of the filtered points
    Z = np.where(~np.isnan(Z), Z + abs(z_threshold), Z)
    # Calculate widths and heights
    widths, heights, max_width, max_height, outlr_idx_h, Ra, Rq, Rp, Rv, Rz, Rsk, Rku = calculate_widths_and_heights(X, Y, Z)  # Get max width and height
    
    mean_width = np.mean(widths)
    std_width = np.std(widths)
    mean_height = np.mean(heights)
    std_height = np.std(heights)

    
    
    # print(f"Mean width: {mean_width}, Std width: {std_width}")
    # print(f"Mean height: {mean_height}, Std height: {std_height}")
    # print(f"Max width: {max_width}, Max height: {max_height}")  # Print max width and height
    stl_file_name = os.path.splitext(os.path.basename(file_path))[0]

#### CONTOUR PLOT WITH LINES
    plt.figure(figsize=(10, 10))
    cp = plt.contour(X, Y, Z, cmap='rainbow', levels=10)
    cbar = plt.colorbar(cp)
    cbar.set_label('Track Height (mm)') 
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title(f'XY Projection of {os.path.basename(file_path)}')
    plt.axis('equal')

    # Existing code for generating contour lines
    min_contour_level = np.nanmin(Z)
    contour_lines = plt.contour(X, Y, Z, levels=[min_contour_level], colors='none')

    # Check if there is at least one collection and one path
    if contour_lines.collections and contour_lines.collections[0].get_paths():
        path = contour_lines.collections[0].get_paths()[0]
        v = path.vertices
        # Continue with the existing logic using `v`
    else:
        # Handle the situation, e.g., by setting v to an empty array or skipping further processing
        v = np.array([])  # Example of setting `v` to an empty array

    # Calculate the bounding box length in the X plane for the thresholded region
    thresholded_points = aligned_points[aligned_points[:, 2] >= z_threshold]
    x_min_thresholded, x_max_thresholded = np.min(thresholded_points[:, 0]), np.max(thresholded_points[:, 0])
    length = x_max_thresholded - x_min_thresholded

    # Add lines and labels for widths, plotting every 5th line
    x_min, x_max = np.min(X), np.max(X)
    total_sections = 30  # Total sections to plot
    section_width = (x_max - x_min) / total_sections
    for i in range(total_sections):
        section_index = i * (len(widths) // total_sections)  # Calculate the index for every 5th section
        width = widths[section_index]
        x_start = x_min + section_index * (x_max - x_min) / len(widths)
        x_end = x_start + section_width

        # Calculate condition
        condition = (X >= x_start) & (X < x_end) & (~np.isnan(Z))
        filtered_Y = Y[condition]

        # Check if the filtered array is empty
        if filtered_Y.size > 0:
            y_min_edge = np.min(filtered_Y)
            y_max_edge = np.max(filtered_Y)
            plt.plot([x_start, x_start], [y_min_edge, y_max_edge], color='black', linestyle='--')
            plt.plot([x_end, x_end], [y_min_edge, y_max_edge], color='black', linestyle='--')
            plt.text((x_start + x_end) / 2, (y_min_edge + y_max_edge) / 2, f'Width: {width:.2f}', rotation=90)

    y_min, y_max = np.min(thresholded_points[:, 1]), np.max(thresholded_points[:, 1])

    # Draw lines representing the bounding box
    plt.plot([x_min_thresholded, x_min_thresholded], [y_min, y_max], 'r--')  # Left line
    plt.plot([x_max_thresholded, x_max_thresholded], [y_min, y_max], 'r--')  # Right line

    # Annotate the length on the plot
    plt.annotate(f'Length: {length:.2f}', 
                xy=((x_min_thresholded + x_max_thresholded) / 2, y_max), 
                textcoords="offset points", 
                xytext=(0,10), 
                ha='center', 
                arrowprops=dict(arrowstyle="->"))

    plt.savefig(f'{stl_file_name}_output_xy_contour.png', dpi=300)
    plt.close()


    # CONTOUR PLOT FILLED
    cp = plt.contourf(X, Y, Z, cmap='rainbow', levels=10)
    cbar = plt.colorbar(cp)
    cbar.set_label('Track Height (mm)') 
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title(f'XY Projection of {os.path.basename(file_path)}')
    plt.axis('equal')
    
    plt.savefig(f'{stl_file_name}_output_xy_contour_FILLED.png', dpi=300)
    plt.close()


  # XZ Projection
    plt.figure(figsize=(10, 10))
    plt.scatter(aligned_points[:, 0], aligned_points[:, 2] + abs(z_threshold), s=0.1, c='black')  # Add z_threshold to Z-values
    plt.xlabel('X')
    plt.ylabel('Z')
    plt.title(f'XZ Projection of {os.path.basename(file_path)}')
    plt.axis('equal')
    plt.savefig(f'{stl_file_name}_output_xz.png', dpi=300)
    plt.close()
    

    # KDE of Z data
    plt.figure(figsize=(10, 10))
    sns.kdeplot(aligned_points[:, 2], fill=True)
    plt.axvline(x=z_threshold, color='red', linestyle='--')
    plt.xlabel('Z')
    plt.title(f'KDE of Z data for {os.path.basename(file_path)}')
    plt.savefig(f'{stl_file_name}_output_kde.png', dpi=300)
    plt.close()


    #filtered KDE Plot to check for gaussian distribution
    # plt.figure(figsize=(10, 10))
    # # Filter aligned_points for Z values above the threshold
    # filtered_z_values = aligned_points[aligned_points[:, 2] >= z_threshold][:, 2]
    # sns.kdeplot(filtered_z_values, fill=True)
    # plt.axvline(x=z_threshold, color='red', linestyle='--')
    # plt.xlabel('Z')
    # plt.title(f'KDE of Filtered Z data for {os.path.basename(file_path)}')
    # plt.savefig(f'{stl_file_name}_output_filtered_kde.png', dpi=300)
    # plt.close()


    data = {
        'stl_file_name': [os.path.basename(file_path)],
        'mean_width': [mean_width],
        'std_width': [std_width],
        'max_width': [max_width],
        'mean_height': [mean_height],
        'outlr_idx_h': [outlr_idx_h],
        'std_height': [std_height],
        'max_height': [max_height],         
        'z_threshold': [z_threshold],  # Add Z threshold
        'volume': [volume],  # Add volume
        'length': [length],  # Add length
        'Ra': [Ra],
        'Rq': [Rq],
        'Rp': [Rp],
        'Rv': [Rv],
        'Rz': [Rz],
        'Rsk': [Rsk],
        'Rku': [Rku]
    }
    df = pd.DataFrame(data)

    return df  # Return max width and height # Return max width and height
file_path = '/Users/ajaytalbot/Library/CloudStorage/OneDrive-UniversityofToronto/STL Files/Single Tracks/TrackSS3.stl'
plot_projections(file_path, grid_size=1000, percent=60)

/var/folders/by/cl20h8994bq9jtq1mdk23hmc0000gn/T/ipykernel_26775/1182618965.py:259: MatplotlibDeprecationWarning: The collections attribute was deprecated in Matplotlib 3.8 and will be removed two minor releases later.
  if contour_lines.collections and contour_lines.collections[0].get_paths():
/var/folders/by/cl20h8994bq9jtq1mdk23hmc0000gn/T/ipykernel_26775/1182618965.py:260: MatplotlibDeprecationWarning: The collections attribute was deprecated in Matplotlib 3.8 and will be removed two minor releases later.
  path = contour_lines.collections[0].get_paths()[0]


,stl_file_name,mean_width,std_width,max_width,mean_height,outlr_idx_h,std_height,max_height,z_threshold,volume,length,Ra,Rq,Rp,Rv,Rz,Rsk,Rku
0,TrackSS3.stl,0.950014,0.118674,1.084954,0.353802,293,0.068237,0.437378,-0.081143,106.520836,10.840828,0.093121,0.277027,0.437378,0.000024,0.437402,-0.672345,2.342336


In [7]:
import os
import pandas as pd  # Ensure pandas is imported

directory = '/Users/ajaytalbot/Library/CloudStorage/OneDrive-UniversityofToronto/STL Files/Single Tracks'
results = []

for filename in os.listdir(directory):
    if filename.endswith(".stl"):
        try:
            file_path = os.path.join(directory, filename)
            result = plot_projections(file_path, grid_size=1000, percent=60)
            results.append(result)
        except Exception as e:
            print(f"Error processing {filename}: {e}")

    df = pd.concat(results, ignore_index=True)



/var/folders/by/cl20h8994bq9jtq1mdk23hmc0000gn/T/ipykernel_26775/1182618965.py:259: MatplotlibDeprecationWarning: The collections attribute was deprecated in Matplotlib 3.8 and will be removed two minor releases later.
  if contour_lines.collections and contour_lines.collections[0].get_paths():
/var/folders/by/cl20h8994bq9jtq1mdk23hmc0000gn/T/ipykernel_26775/1182618965.py:260: MatplotlibDeprecationWarning: The collections attribute was deprecated in Matplotlib 3.8 and will be removed two minor releases later.
  path = contour_lines.collections[0].get_paths()[0]
/var/folders/by/cl20h8994bq9jtq1mdk23hmc0000gn/T/ipykernel_26775/1182618965.py:259: MatplotlibDeprecationWarning: The collections attribute was deprecated in Matplotlib 3.8 and will be removed two minor releases later.
  if contour_lines.collections and contour_lines.collections[0].get_paths():
/var/folders/by/cl20h8994bq9jtq1mdk23hmc0000gn/T/ipykernel_26775/1182618965.py:260: MatplotlibDeprecationWarning: The collections attrib

In [8]:
df_geom=df

In [9]:
df_geom

,stl_file_name,mean_width,std_width,max_width,mean_height,outlr_idx_h,std_height,max_height,z_threshold,volume,length,Ra,Rq,Rp,Rv,Rz,Rsk,Rku
0,Track9.stl,0.830789,0.093880,0.946779,0.198121,602,0.048837,0.274716,-0.052163,18.778051,10.575848,0.058161,0.150720,0.274716,0.000014,0.274730,-0.189105,2.057495
1,TrackSS21.stl,0.863256,0.091871,0.950242,0.445514,157,0.071095,0.496298,-0.097529,29.944098,10.672044,0.095844,0.371376,0.496298,0.000615,0.496914,-1.214499,3.601690
2,TrackSS35.stl,1.049849,0.133119,1.154380,0.483624,157,0.088099,0.551265,-0.164682,160.370421,10.666943,0.105937,0.403602,0.551265,0.000101,0.551365,-1.083703,3.333452
3,Track15.stl,0.913643,0.118914,1.281946,0.272916,495,0.053759,0.374157,-0.069800,16.973001,10.908054,0.072236,0.210692,0.374157,0.000012,0.374169,-0.362616,2.369195
4,Track29.stl,0.885407,0.095562,1.024702,0.227112,572,0.045404,0.314887,-0.063285,61.845191,10.677670,0.062860,0.173320,0.314887,0.000063,0.314950,-0.349188,2.172694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,Track24.stl,0.943966,0.108175,1.017152,0.424274,382,0.082023,0.493173,-0.138023,76.891976,10.692512,0.105115,0.345753,0.493173,0.000174,0.493347,-0.860856,2.701697
116,Track18.stl,0.786368,0.111216,1.571119,0.183800,634,0.042938,0.260749,-0.029368,24.209671,10.555745,0.052139,0.141798,0.260749,0.000002,0.260751,-0.134815,2.224697
117,TrackSS38.stl,0.957328,0.116166,1.064750,0.368053,152,0.065992,0.421525,-0.079425,146.107009,10.685210,0.088757,0.295816,0.421525,0.000295,0.421820,-0.895900,2.746738
118,TrackSS10.stl,0.871662,0.091890,0.965638,0.390640,176,0.063768,0.442626,-0.085149,61.726204,10.652258,0.092864,0.316347,0.442626,0.000212,0.442837,-1.009157,2.948823


In [13]:
#separate the dataframe into 3 data frames for ones that have file name Track, TrackSS, and TrackINSS
df_track = df_geom[df_geom['stl_file_name'].str.contains('Track')]
df_trackSS = df_geom[df_geom['stl_file_name'].str.contains('TrackSS')]
df_trackINSS = df_geom[df_geom['stl_file_name'].str.contains('TrackINSS')]



In [15]:
#delete TrackSS from the file name and then reorder in ascending order based on the file name number
df_trackSS['Material'] = 'SS316'
df_trackSS['stl_file_name'] = df_trackSS['stl_file_name'].str.replace('TrackSS', '')
df_trackSS['stl_file_name'] = df_trackSS['stl_file_name'].str.replace('.stl', '')
df_trackSS['stl_file_name'] = df_trackSS['stl_file_name'].astype(int)
df_trackSS = df_trackSS.sort_values(by='stl_file_name', ascending=True)
df_trackSS

/var/folders/by/cl20h8994bq9jtq1mdk23hmc0000gn/T/ipykernel_26775/1833208081.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trackSS['Material'] = 'SS316'
/var/folders/by/cl20h8994bq9jtq1mdk23hmc0000gn/T/ipykernel_26775/1833208081.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trackSS['stl_file_name'] = df_trackSS['stl_file_name'].str.replace('TrackSS', '')
/var/folders/by/cl20h8994bq9jtq1mdk23hmc0000gn/T/ipykernel_26775/1833208081.py:4: SettingWithCopyWarning: 
A value is trying to be set on 

,stl_file_name,mean_width,std_width,max_width,mean_height,outlr_idx_h,std_height,max_height,z_threshold,volume,length,Ra,Rq,Rp,Rv,Rz,Rsk,Rku,Material
76,1,0.790831,0.092920,1.279393,0.303548,424,0.060610,0.357561,0.001778,118.911095,10.593417,0.083471,0.234332,0.357561,3.666603e-03,0.361228,-0.585784,2.130991,SS316
83,2,0.925072,0.118035,1.032043,0.416560,265,0.079554,0.484661,-0.102807,144.371180,10.658857,0.095619,0.344188,0.484661,2.751393e-04,0.484936,-1.024156,3.095645,SS316
82,3,0.950014,0.118674,1.084954,0.353802,293,0.068237,0.437378,-0.081143,106.520836,10.840828,0.093121,0.277027,0.437378,2.412842e-05,0.437402,-0.672345,2.342336,SS316
67,4,0.863495,0.091736,0.961026,0.294201,299,0.051950,0.371784,-0.070105,81.140554,10.682586,0.077474,0.228785,0.371784,4.288650e-05,0.371827,-0.710202,2.353308,SS316
64,5,0.944876,0.115240,1.049966,0.426658,264,0.081702,0.489502,-0.117823,109.200337,10.687982,0.101018,0.348899,0.489502,1.488933e-04,0.489651,-0.971177,2.918218,SS316
57,6,0.836232,0.107476,1.298503,0.259218,371,0.046886,0.327457,-0.064035,75.437102,10.515113,0.068733,0.200795,0.327457,3.151711e-04,0.327772,-0.582281,2.320183,SS316
60,7,0.874813,0.080138,0.942680,0.307802,295,0.053427,0.360269,-0.073276,64.579807,10.732797,0.080990,0.239554,0.360269,1.807074e-04,0.360450,-0.742977,2.367416,SS316
48,8,0.964285,0.116143,1.081988,0.396526,338,0.077874,0.472223,-0.098714,77.554948,10.823765,0.103346,0.316015,0.472223,4.658372e-04,0.472689,-0.741405,2.447455,SS316
47,9,0.791152,0.114751,0.887174,0.323259,307,0.065227,0.386974,-0.078707,56.804320,10.681582,0.083871,0.259971,0.386974,3.193373e-05,0.387006,-0.818402,2.526020,SS316
118,10,0.871662,0.091890,0.965638,0.390640,176,0.063768,0.442626,-0.085149,61.726204,10.652258,0.092864,0.316347,0.442626,2.115454e-04,0.442837,-1.009157,2.948823,SS316


In [14]:
#df_track is ythe dataframe of all the rows minus the ones that contain TrackSS and TrackINSS
df_track = df_geom[~df_geom['stl_file_name'].str.contains('TrackSS')]
df_track = df_track[~df_track['stl_file_name'].str.contains('TrackINSS')]
#strip Track from the file name and then reoder in ascending order based on the file name number 
#delet the row that has track 1 
#new column called Material with the value of 'IN625' 
df_track['Material'] = 'IN625'
df_track = df_track[~df_track['stl_file_name'].str.contains('track 1')]
df_track['stl_file_name'] = df_track['stl_file_name'].str.replace('Track', '')
df_track['stl_file_name'] = df_track['stl_file_name'].str.replace('.stl', '')
df_track['stl_file_name'] = df_track['stl_file_name'].astype(int)
df_track = df_track.sort_values(by='stl_file_name')
df_track

,stl_file_name,mean_width,std_width,max_width,mean_height,outlr_idx_h,std_height,max_height,z_threshold,volume,length,Ra,Rq,Rp,Rv,Rz,Rsk,Rku,Material
93,1,0.715689,0.088251,1.091696,0.164563,376,0.035110,0.228636,-0.004600,47.577736,10.420077,0.048002,0.123949,0.228636,0.000048,0.228685,-0.269835,2.060748,IN625
100,2,0.807450,0.167129,0.969302,0.235381,531,0.081535,0.362145,-0.036660,60.838743,11.990442,0.074894,0.194190,0.362145,0.000006,0.362151,-0.118610,2.114070,IN625
94,3,0.797756,0.114674,1.084164,0.151327,622,0.060084,0.282299,-0.019708,37.131865,10.600752,0.051621,0.123150,0.282299,0.000003,0.282301,0.631344,2.786181,IN625
119,4,0.844248,0.166668,1.535752,0.158050,456,0.033819,0.253342,-0.019652,35.247759,10.632828,0.043602,0.118182,0.253342,0.000164,0.253505,-0.169988,2.315906,IN625
109,5,0.881068,0.098107,1.003947,0.260227,447,0.063271,0.393242,-0.053076,44.465158,10.664553,0.074860,0.203232,0.393242,0.000325,0.393567,-0.187800,2.228513,IN625
101,6,0.858941,0.119250,1.196102,0.184958,535,0.045297,0.261156,-0.048440,28.165308,10.706003,0.053766,0.140208,0.261156,0.000014,0.261170,-0.184730,2.066215,IN625
108,7,0.818248,0.113577,1.339122,0.129922,644,0.043483,0.221891,-0.017614,21.882695,10.719327,0.040364,0.099731,0.221891,0.000011,0.221902,0.383536,2.500578,IN625
8,8,0.900085,0.116719,1.385540,0.223726,493,0.048543,0.301748,-0.043486,26.586315,10.645484,0.062821,0.168885,0.301748,0.000022,0.301770,-0.315538,2.092678,IN625
0,9,0.830789,0.093880,0.946779,0.198121,602,0.048837,0.274716,-0.052163,18.778051,10.575848,0.058161,0.150720,0.274716,0.000014,0.274730,-0.189105,2.057495,IN625
25,10,0.867212,0.093709,0.992910,0.233445,461,0.051126,0.324929,-0.078335,16.441515,10.588707,0.065756,0.177872,0.324929,0.000150,0.325080,-0.290669,2.155634,IN625


In [16]:
#delete TrackINSS from the file name and then reorder in ascending order based on the file name number
df_trackINSS['Material'] = 'IN625+SS316'
df_trackINSS['stl_file_name'] = df_trackINSS['stl_file_name'].str.replace('TrackINSS', '')
df_trackINSS['stl_file_name'] = df_trackINSS['stl_file_name'].str.replace('.stl', '')
df_trackINSS['stl_file_name'] = df_trackINSS['stl_file_name'].astype(int)
df_trackINSS = df_trackINSS.sort_values(by='stl_file_name')
df_trackINSS

/var/folders/by/cl20h8994bq9jtq1mdk23hmc0000gn/T/ipykernel_26775/1338766372.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trackINSS['Material'] = 'IN625+SS316'
/var/folders/by/cl20h8994bq9jtq1mdk23hmc0000gn/T/ipykernel_26775/1338766372.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trackINSS['stl_file_name'] = df_trackINSS['stl_file_name'].str.replace('TrackINSS', '')
/var/folders/by/cl20h8994bq9jtq1mdk23hmc0000gn/T/ipykernel_26775/1338766372.py:4: SettingWithCopyWarning: 
A value is trying

,stl_file_name,mean_width,std_width,max_width,mean_height,outlr_idx_h,std_height,max_height,z_threshold,volume,length,Ra,Rq,Rp,Rv,Rz,Rsk,Rku,Material
70,1,1.019871,0.446001,1.980497,0.286934,559,0.076344,0.467990,0.001053,190.015205,10.675822,0.085686,0.216833,0.467990,0.002161,0.470151,0.070532,2.329528,IN625+SS316
55,3,0.925639,0.134888,1.090392,0.345712,541,0.095801,0.536126,-0.119604,96.055266,11.082881,0.101816,0.279408,0.536126,0.000075,0.536201,-0.208144,2.268415,IN625+SS316
79,4,0.886018,0.104545,1.002796,0.308179,607,0.071854,0.409708,-0.126684,78.149164,10.804735,0.084644,0.242654,0.409708,0.000077,0.409785,-0.361501,2.241051,IN625+SS316
71,5,0.889282,0.100360,0.994975,0.421933,584,0.089033,0.523028,-0.153938,105.680097,10.731381,0.106442,0.346222,0.523028,0.000055,0.523082,-0.754784,2.618650,IN625+SS316
80,6,0.819860,0.091234,1.001057,0.279620,261,0.057985,0.404828,-0.095751,70.654258,10.677285,0.071257,0.222377,0.404828,0.000039,0.404868,-0.455580,2.684269,IN625+SS316
85,7,0.849518,0.104017,0.946376,0.267392,398,0.057633,0.378187,-0.090788,56.746480,10.766228,0.071466,0.210770,0.378187,0.000295,0.378482,-0.471631,2.392174,IN625+SS316
44,8,0.940965,0.109769,1.304744,0.350559,520,0.132605,0.681914,-0.158927,64.456542,11.013140,0.116538,0.299038,0.681914,0.000146,0.682060,0.550903,3.071582,IN625+SS316
35,9,0.783407,0.083335,1.140106,0.286143,475,0.051143,0.383244,-0.101434,50.019924,10.707578,0.075673,0.224138,0.383244,0.000096,0.383340,-0.566507,2.325398,IN625+SS316
68,10,0.857683,0.102767,0.983566,0.371121,486,0.080784,0.490756,-0.135703,53.951266,10.659584,0.096735,0.302197,0.490756,0.000225,0.490981,-0.582281,2.501348,IN625+SS316
63,11,0.891707,0.110184,1.016850,0.294710,647,0.077294,0.434137,-0.098135,39.692517,10.803237,0.082451,0.237553,0.434137,0.000100,0.434237,-0.227790,2.341451,IN625+SS316


In [17]:
#now we can combine all the dataframes into one
df_COMB_EXPRT = pd.concat([df_track, df_trackSS, df_trackINSS])

In [19]:
#export the dataframes to csv files
df_COMB_EXPRT.to_csv('df_1000_COMB.csv', index=False)
# df_trackSS.to_csv('df_trackSS_4.csv', index=False)
# df_trackINSS.to_csv('df_trackINSS_4.csv', index=False)
